In [1]:
! pip install -r requirements.txt
! pip install -r /home/sojungkim2/legalmaster/LegalMaster/demo/requirements.txt

  Using cached transformers-4.29.1-py3-none-any.whl (7.1 MB)
  Using cached peft-0.3.0-py3-none-any.whl (56 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0.dev0
    Uninstalling transformers-4.31.0.dev0:
      Successfully uninstalled transformers-4.31.0.dev0
^C
ERROR: Operation cancelled by user
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-pwl3g7i9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-pwl3g7i9
  Resolved https://github.com/huggingface/peft.git to commit f4526d57fc26e3a1f592da4f3bfac9816c335256
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pmg9uaq2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-pmg9

  Resolved https://github.com/huggingface/transformers.git to commit 12240925cfa29fff932e49927eb9744713ab1018
  Installing build dependencies ... 

In [1]:
import argparse
import logging
import random
import pickle
import tqdm
import shortuuid
import pandas as pd
import numpy as np
import os
import bitsandbytes as bn


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
## models
import torch
import torch.nn as nn
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel
from utils.model import load_tokenizer_and_model, load_tokenizer_and_model_multiple

### Base 모델에 학습완료한 adapter를 merge하여 parameters 보기

##### Chat

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [4]:
base_model_path = '/home/sojungkim2/legalmaster/7Boutput'
#adapter_path = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
adapter_model_path = '/home/sojungkim2/legalmaster/LegalMaster/ChatAdapterTraining/checkpoints'

In [5]:
tokenizer, model, _device = load_tokenizer_and_model(base_model_path, adapter_model_path, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0096, -0.0301,  0.0085,  

In [10]:
[t for t in model.base_model.model.model.layers[2].mlp.gate_proj.lora_B.parameters()]

[Parameter containing:
 tensor([[-1.6500e-03, -1.4201e-03, -3.5731e-03,  ..., -8.3206e-04,
           9.6930e-04,  8.1540e-04],
         [-1.3900e-03, -7.0310e-04, -2.1367e-03,  ..., -4.0451e-04,
           1.6802e-03, -2.7458e-03],
         [-3.0124e-03, -2.3920e-03, -9.7753e-04,  ..., -1.1115e-03,
           2.2438e-03, -3.0881e-03],
         ...,
         [-4.7411e-03, -4.3072e-05, -8.2167e-03,  ..., -1.8420e-03,
           1.6241e-03,  5.9094e-04],
         [-1.5378e-03, -2.8751e-03,  1.6084e-03,  ..., -2.3933e-03,
           3.6613e-03, -1.4755e-03],
         [-1.1156e-04,  1.4043e-03, -3.5875e-03,  ...,  2.0503e-04,
          -7.4876e-04, -3.0832e-05]])]

##### Legal

In [11]:
logger_1 = logging.getLogger(__name__)
logger_1.setLevel(logging.INFO)

In [12]:
base_model_path_1 = '/home/sojungkim2/legalmaster/7Boutput'
#adapter_path_1 = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
adapter_model_path_1 = '/home/sojungkim2/legalmaster/LegalMaster/LegalAdapterTraining/checkpoints'


In [13]:
tokenizer_1, model_1, _device_1 = load_tokenizer_and_model(base_model_path_1, adapter_model_path_1, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
model_1.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0096, -0.0301,  0.0085,  

In [16]:
[t for t in model_1.base_model.model.model.layers[2].mlp.gate_proj.lora_B.parameters()]

[Parameter containing:
 tensor([[-0.0012,  0.0052, -0.0058,  ..., -0.0066,  0.0007,  0.0044],
         [ 0.0054,  0.0027, -0.0018,  ..., -0.0061, -0.0045,  0.0028],
         [ 0.0063,  0.0033, -0.0038,  ..., -0.0056, -0.0070,  0.0019],
         ...,
         [ 0.0073,  0.0066, -0.0072,  ..., -0.0065, -0.0028,  0.0069],
         [-0.0077, -0.0037,  0.0060,  ...,  0.0082, -0.0004, -0.0057],
         [-0.0040,  0.0006, -0.0028,  ..., -0.0005,  0.0083,  0.0004]])]

### Chat + Legal - load_tokenizer_and_model_multiple

In [3]:
logger_2 = logging.getLogger(__name__)
logger_2.setLevel(logging.INFO)

In [4]:
base_model_path_2 = '/home/sojungkim2/legalmaster/MergeModels/base_chat'
adapter_path_1 = '/home/sojungkim2/legalmaster/LegalMaster/ChatAdapterTraining/checkpoints'
adapter_path_2 = '/home/sojungkim2/legalmaster/LegalMaster/LegalAdapterTraining/checkpoints'


In [5]:
tokenizer_2, model_2, _device_2 = load_tokenizer_and_model_multiple(base_model_path_2, adapter_path_1, adapter_path_2, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model_2.state_dict()

OrderedDict([('base_model.model.base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.base_model.model.model.layers.0.self_attn.q_proj.weight',
              ten

In [7]:
[t for t in model_2.base_model.base_model.model.model.layers[2].mlp.gate_proj.lora_B.parameters()]

[Parameter containing:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])]

In [8]:
model_2

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(32000, 4096, padding_idx=0)
            (layers): ModuleList(
              (0-31): 32 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): Linear(
                    in_features=4096, out_features=4096, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embeddin

### Chat + Legal - load merged

In [15]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import PeftModel
import transformers
from typing import Iterator

def load_merged_model(model_path, load_8bit=False):
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"

    try:
        if torch.backends.mps.is_available():
            device = "mps"
    except:  # noqa: E722
        pass
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    if device == "cuda":
        model = LlamaForCausalLM.from_pretrained(
            model_path,
            load_in_8bit=load_8bit,
            torch_dtype=torch.float16,
            device_map={"": 0},
        )

    elif device == "mps":
        model = LlamaForCausalLM.from_pretrained(
            model_path,
            device_map={"": device},
            torch_dtype=torch.float16,
        )
 
    else:
        model = LlamaForCausalLM.from_pretrained(
            model_path, device_map={"": device}, low_cpu_mem_usage=True
        )

    if not load_8bit and device != "cpu":
        model.half()  # seems to fix bugs for some users.

    model.eval()
    return tokenizer, model, device

In [11]:
logger_3 = logging.getLogger(__name__)
logger_3.setLevel(logging.INFO)

In [12]:
merged_model_path_chat_legal = '/home/sojungkim2/legalmaster/LegalMaster/MergeModels/base_chat_legal'

In [13]:
tokenizer_3, model_3, _device_3 = load_tokenizer_and_model(merged_model_path_chat_legal,load_8bit=True)

TypeError: load_tokenizer_and_model() missing 1 required positional argument: 'adapter_path'

In [16]:
tokenizer_3, model_3, _device_3 = load_merged_model(merged_model_path_chat_legal,load_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
model_3.state_dict()

OrderedDict([('model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0097, -0.0300,  0.0085,  ...,  0.0182, -0.0049, -0.0363],
 

In [21]:
[t for t in model_3.model.layers[2].mlp.gate_proj.parameters()]

[Parameter containing:
 tensor([[-0.0074, -0.0097, -0.0054,  ...,  0.0159, -0.0478, -0.0274],
         [ 0.0515,  0.0020,  0.0410,  ..., -0.0215,  0.0064, -0.0054],
         [ 0.0237,  0.0031,  0.0188,  ...,  0.0208, -0.0025,  0.0396],
         ...,
         [-0.0009, -0.0449, -0.0746,  ..., -0.0147,  0.0382,  0.0044],
         [ 0.0301, -0.0021, -0.0092,  ..., -0.0048,  0.0042, -0.0096],
         [ 0.0507,  0.0225, -0.0040,  ..., -0.0072,  0.0163, -0.0105]],
        requires_grad=True)]

In [19]:
model_3

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

### Chat + Legal  - load_tokenizer_and_model ((llama + chat) + legal) 

In [22]:
logger_4 = logging.getLogger(__name__)
logger_4.setLevel(logging.INFO)

In [23]:
base_model_path = '/home/sojungkim2/legalmaster/MergeModels/base_chat'
adapter_path_1 = '/home/sojungkim2/legalmaster/LegalMaster/LegalAdapterTraining/checkpoints'


In [24]:
tokenizer_4, model_4, _device_4 = load_tokenizer_and_model(base_model_path, adapter_path_1, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
model_4.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0097, -0.0300,  0.0084,  

In [30]:
[t for t in model_4.base_model.model.model.layers[2].mlp.gate_proj.lora_B.parameters()]

[Parameter containing:
 tensor([[-0.0012,  0.0052, -0.0058,  ..., -0.0066,  0.0007,  0.0044],
         [ 0.0054,  0.0027, -0.0018,  ..., -0.0061, -0.0045,  0.0028],
         [ 0.0063,  0.0033, -0.0038,  ..., -0.0056, -0.0070,  0.0019],
         ...,
         [ 0.0073,  0.0066, -0.0072,  ..., -0.0065, -0.0028,  0.0069],
         [-0.0077, -0.0037,  0.0060,  ...,  0.0082, -0.0004, -0.0057],
         [-0.0040,  0.0006, -0.0028,  ..., -0.0005,  0.0083,  0.0004]])]

In [29]:
model_4.base_model.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
          

In [27]:
model_4

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
        